In [0]:
%run
./_common_helpers

---
### Purpose: ###  
Feed raw EHR bronze data into silver `claim_headers`, `claim_lines` tables


---
- Normalize types
- Map EHR `workqueue_status` -> claim `current_status`
- Treat each EHR row as one line; if later on you ingest line-item detail it will remain compatible.
- Bad rows missing `claim_id`, bad money/date go to the `_rejects` table
---

---
### Assumes: ###  
`as_of_date` is the claim submission snapshot date from the EHR. 

In [0]:
------------ EHR staging layer (typed, validated, dedup) ----------
CREATE OR REPLACE TEMP VIEW stage_claim_headers AS
WITH base AS (
  SELECT
    upper(trim(claim_id))   AS claim_id,
    upper(trim(payer_id))   AS payer_id,
    upper(trim(patient_id)) AS patient_id,
    silver.fn_to_date_safe(as_of_date)  AS submission_date,
    silver.fn_to_ts_safe(as_of_date)    AS submission_ts,
    silver.fn_to_dec_safe(billed_amt)   AS total_charges,
    silver.fn_to_dec_safe(expected_amt) AS expected_amount,
    upper(trim(workqueue_status)) AS wq_status,
    _ingest_ts, _ingest_file
  FROM bronze.ehr_claims_raw
  WHERE claim_id IS NOT NULL
    AND silver.fn_to_dec_safe(billed_amt) IS NOT NULL
    AND silver.fn_to_date_safe(as_of_date) IS NOT NULL
)
SELECT
  claim_id, payer_id, patient_id,
  submission_date, submission_ts,
  total_charges, expected_amount,
  CASE
    WHEN wq_status IN ('SUBMITTED','RESUBMITTED') THEN 'Submitted'
    WHEN wq_status IN ('IN WQ','PENDING REVIEW')  THEN 'Pending'
    WHEN wq_status = 'DENIED'                     THEN 'Denied'
    ELSE 'Pending'
  END AS current_status,
  submission_ts AS current_status_ts,
  CAST(NULL AS TIMESTAMP) AS last_277ca_event_ts,
  CAST(NULL AS TIMESTAMP) AS last_835_event_ts,
  dayofweek(submission_date) AS weekday_submitted,
  dayofweek(submission_date) IN (1,7) AS is_weekend_submission,
  'EHR' AS source_system,
  _ingest_ts, _ingest_file,
  ROW_NUMBER() OVER (PARTITION BY claim_id ORDER BY submission_ts DESC, _ingest_ts DESC) AS rn
FROM base
QUALIFY rn = 1;

In [0]:
------------ Target table for claim headers (SCD1 current view) ----------
CREATE OR REPLACE TABLE silver.claim_headers (
  claim_id STRING NOT NULL,
  payer_id STRING,
  patient_id STRING,
  submission_date DATE,
  submission_ts TIMESTAMP,
  total_charges DECIMAL(18,2),
  expected_amount DECIMAL(18,2),
  current_status STRING,
  current_status_ts TIMESTAMP,
  last_277ca_event_ts TIMESTAMP,
  last_835_event_ts TIMESTAMP,
  weekday_submitted INT,
  is_weekend_submission BOOLEAN,
  source_system STRING,
  _ingest_ts TIMESTAMP,
  _ingest_file STRING,
  CONSTRAINT submission_date_nn EXPECT (submission_date IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT amounts_non_negative EXPECT ( (total_charges IS NULL OR total_charges >= 0) AND expected_amount >= 0 ),
  CONSTRAINT expected_amount_nn EXPECT (expected_amount IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT check_current_status EXPECT (upper(current_status) IN ('SUBMITTED','PENDING','DENIED')),
  CONSTRAINT source_is_ehr EXPECT (source_system = 'EHR')
)
USING DELTA
CLUSTER BY (claim_id);

------------ Type-1 upsert (latest wins by current_status_ts, then _ingest_ts) ----------
MERGE INTO silver.claim_headers t
USING stage_claim_headers s
ON t.claim_id = s.claim_id
WHEN MATCHED AND (
      s.current_status_ts >  t.current_status_ts
   OR (s.current_status_ts = t.current_status_ts AND s._ingest_ts > t._ingest_ts)
) THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;

In [0]:
------------ Rejections for observability ----------
CREATE OR REPLACE TABLE silver.ehr_claims_rejects AS
SELECT
  r.*,
  CASE
    WHEN claim_id IS NULL THEN 'MISSING_CLAIM_ID'
    WHEN silver.fn_to_dec_safe(billed_amt) IS NULL THEN 'BAD_BILLED_AMT'
    WHEN silver.fn_to_dec_safe(expected_amt) IS NULL THEN 'BAD_EXPECTED_AMT'
    WHEN silver.fn_to_date_safe(as_of_date) IS NULL THEN 'BAD_DATE'
    ELSE 'OTHER'
  END AS reject_reason
FROM bronze.ehr_claims_raw r
WHERE claim_id IS NULL
   OR silver.fn_to_dec_safe(billed_amt) IS NULL
   OR silver.fn_to_dec_safe(expected_amt) IS NULL
   OR silver.fn_to_date_safe(as_of_date) IS NULL;

In [0]:
------------ Claim lines (one line per EHR row) ----------
CREATE OR REPLACE TABLE silver.claim_lines AS
SELECT 
  upper(trim(r.claim_id))          AS claim_id,
  1                                AS line_number,
  CAST(NULL as STRING)             AS rev_code,
  upper(trim(cpt))                 AS hcpcs_cpt,
  CAST(array() AS ARRAY<STRING>)   AS modifier_codes,
  CAST(1 AS DECIMAL(18,2))         AS units,
  silver.fn_to_dec_safe(billed_amt)AS line_charge,
  CAST(NULL AS STRING)             AS rendering_npi,
  CAST(array(1) AS ARRAY<INT>)     AS dx_pointer,
  (silver.fn_to_dec_safe(billed_amt) >= 5000) AS is_high_cost,
  'EHR'                            AS source_system,
  _ingest_ts, _ingest_file
FROM bronze.ehr_claims_raw r
WHERE r.claim_id IS NOT NULL
  AND silver.fn_to_dec_safe(billed_amt) IS NOT NULL
  AND silver.fn_to_date_safe(as_of_date) IS NOT NULL;